In [1]:
import numpy as np
import pandas as pd
# plots
import seaborn as sns; sns.set_theme()
sns.set(palette='colorblind')
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import OrdinalEncoder

Alexandre Dias Negretti	233609	
Daniel Godoy Marques	166213	
Gyovana Mayara Moriyama	216190	

In [2]:
path = '../data/processed/'
df_train = pd.read_csv(path + 'scenario03.csv', sep=',')
df_test = pd.read_csv(path + 'scenario04.csv', sep=',')

Train -> scenario03

Test -> scenario04

# Dealing with Categorical Features - Encoding

In [ ]:
cols = ['organization', 'age', 'cnt_encounters', 'cts', 'contraceptive', 'anticoagulant', 'cnt_medications', 'cnt_procedures', 'last_encounter_dur', 'condition_dur', 
        'gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient', 'target']

In [ ]:
df_train = pd.get_dummies(df_train, columns=['gender'], drop_first=True)
df_train = pd.get_dummies(df_train, columns=['encounterclass']) 
df_train['encounterclass_emergency'] = 0
df_train = df_train[cols]

df_test = pd.get_dummies(df_test, columns=['gender'], drop_first=True)
df_test = pd.get_dummies(df_test, columns=['encounterclass']) 
df_test = df_test[cols]
df_train.head()

,organization,age,cnt_encounters,cts,contraceptive,anticoagulant,cnt_medications,cnt_procedures,last_encounter_dur,condition_dur,gender_M,encounterclass_ambulatory,encounterclass_emergency,encounterclass_inpatient,encounterclass_outpatient,target
0,NORTHWEST HOSPITAL,52,38,0,0,0,0,0,6,389,1,0,0,1,0,1
1,VALLEY MEDICAL CENTER,74,40,0,0,0,0,0,0,386,1,0,0,0,1,1
2,KITTITAS VALLEY COMMUNITY HOSPITAL,33,12,0,0,0,0,0,10,1,1,0,0,1,0,0
3,Walla Walla County Vet Center,58,27,0,0,0,0,0,12,4,1,0,0,1,0,0
4,ST FRANCIS COMMUNITY HOSPITAL,58,28,0,0,0,0,0,10,4,0,0,0,1,0,0


In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=85)
encoder.fit(df_train[['organization']])
df_train[['organization']] = encoder.transform(df_train[['organization']])

df_test[['organization']] = encoder.transform(df_test[['organization']])

In [ ]:
df_train

,organization,age,cnt_encounters,cts,contraceptive,anticoagulant,cnt_medications,cnt_procedures,last_encounter_dur,condition_dur,gender_M,encounterclass_ambulatory,encounterclass_emergency,encounterclass_inpatient,encounterclass_outpatient,target
0,13.0,52,38,0,0,0,0,0,6,389,1,0,0,1,0,1
1,32.0,74,40,0,0,0,0,0,0,386,1,0,0,0,1,1
2,6.0,33,12,0,0,0,0,0,10,1,1,0,0,1,0,0
3,34.0,58,27,0,0,0,0,0,12,4,1,0,0,1,0,0
4,26.0,58,28,0,0,0,0,0,10,4,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,27.0,33,13,0,0,0,0,0,9,3,1,0,0,1,0,0
66,25.0,97,79,0,0,0,0,0,5,669,1,0,0,1,0,1
67,4.0,44,35,0,0,0,0,0,10,0,0,0,0,1,0,0
68,35.0,30,22,0,0,0,0,0,13,11,1,0,0,1,0,0


# Models

## Split

In [ ]:
X_train = df_train.drop(['target'], axis=1)
y_train = df_train['target']

X_test = df_test.drop(['target'], axis=1)
y_test = df_test['target']

In [ ]:
X_test

,organization,age,cnt_encounters,cts,contraceptive,anticoagulant,cnt_medications,cnt_procedures,last_encounter_dur,condition_dur,gender_M,encounterclass_ambulatory,encounterclass_emergency,encounterclass_inpatient,encounterclass_outpatient
0,85.0,69,51,0,0,2,10,0,9,6,0,0,0,1,0
1,85.0,65,26,0,0,2,10,0,12,0,1,0,0,1,0
2,85.0,32,11,0,0,2,5,0,10,0,1,0,0,1,0
3,85.0,53,43,0,2,2,16,0,12,2,0,0,0,1,0
4,85.0,59,25,0,0,2,9,0,9,3,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,85.0,10,7,0,0,2,8,0,1,1,1,0,0,1,0
626,85.0,80,25,0,0,3,17,0,11,459,1,0,0,1,0
627,85.0,51,38,0,0,2,9,0,11,5,0,0,0,1,0
628,85.0,21,10,0,0,2,6,0,10,4,1,0,0,1,0


## SMOTE

In [ ]:
oversample = SMOTE()
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

## Model Baseline

In [ ]:
predictions = np.zeros(len(y_train), dtype=int)
y_prob_n = np.zeros(len(y_train), dtype=int)

print(f'Accuracy: {accuracy_score(y_train.values, predictions)}')
print(f'AUC: {roc_auc_score(y_train.values, y_prob_n)}')
print(f'F1-Score: {f1_score(y_train.values,predictions)}')

Accuracy: 0.8
AUC: 0.5
F1-Score: 0.0


In [ ]:
predictions = np.zeros(len(y_train_over), dtype=int)
y_prob_n = np.zeros(len(y_train_over), dtype=int)

print(f'Accuracy: {accuracy_score(y_train_over.values, predictions)}')
print(f'AUC: {roc_auc_score(y_train_over.values, y_prob_n)}')
print(f'F1-Score: {f1_score(y_train_over.values, predictions)}')

Accuracy: 0.5
AUC: 0.5
F1-Score: 0.0


## Creating models


In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import time

In [ ]:
def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame) -> None:
  models = [
            ('LogReg', LogisticRegression(max_iter=500)), 
            ('RF', RandomForestClassifier(random_state=42)),
            ('LGBM', LGBMClassifier(random_state=42)),
            ('XGB', XGBClassifier(random_state=42))
          ]

  for name, model in models:
    print(f'{name}:')
    results = pd.DataFrame()

    start = time.time()
    clf = model.fit(X_train, y_train)
    end = time.time()
    results.loc[0,'Fit time'] = end - start

    start = time.time()
    y_pred = clf.predict(X_test)
    end = time.time()
    results.loc[0,'Score time'] = end - start
    y_proba = clf.predict_proba(X_test)

    results.loc[0,'Accuracy'] = accuracy_score(y_test.values, y_pred)
    results.loc[0, 'AUC'] = roc_auc_score(y_test.values, y_proba[:, 1])
    results.loc[0, 'F1-Score'] = f1_score(y_test.values, y_pred)
    display(results)
    print()

    if name == 'XGB':
      print()
      for feature, imp in zip(X_train.columns, clf.feature_importances_):
        print(f'{feature:<25} {imp}')


### Imbalanced and using all features

In [ ]:
run_exps(X_train, y_train, X_test, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.016677,0.001498,0.996825,0.992171,0.992593



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.168975,0.016102,0.977778,0.998244,0.945736



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.011928,0.003112,0.912698,0.970878,0.767932



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.014862,0.00149,0.996825,0.991575,0.992593




organization              0.0
age                       0.0
cnt_encounters            0.0
cts                       0.0
contraceptive             0.0
anticoagulant             0.0
cnt_medications           0.0
cnt_procedures            0.0
last_encounter_dur        0.0
condition_dur             1.0
gender_M                  0.0
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### Imbalanced and using just features from original tables


In [ ]:
X_train_og = X_train[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()
X_test_og = X_test[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()

run_exps(X_train_og, y_train, X_test_og, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.007931,0.002292,0.784127,0.559605,0.0



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.138981,0.015163,0.784127,0.549484,0.0



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.012073,0.003841,0.784127,0.5593,0.0



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.01539,0.001228,0.784127,0.5593,0.0




gender_M                  1.0
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### Imbalanced and removing condition_dur feature

In [ ]:
X_train_c = X_train.drop('condition_dur', axis=1)
X_test_c = X_test.drop('condition_dur', axis=1)

run_exps(X_train_c, y_train, X_test_c, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.042358,0.001528,0.249206,0.927914,0.365101



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.134543,0.016218,0.844444,0.861835,0.608



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.013522,0.003128,0.792063,0.818126,0.584127



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.016697,0.002124,0.84127,0.865906,0.64539




organization              0.2418103814125061
age                       0.06115444377064705
cnt_encounters            0.3282798230648041
cts                       0.0
contraceptive             0.0
anticoagulant             0.0
cnt_medications           0.0
cnt_procedures            0.0
last_encounter_dur        0.2684614360332489
gender_M                  0.10029397159814835
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### With SMOTE and using all features

In [ ]:
run_exps(X_train_over, y_train_over, X_test, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.024632,0.008995,0.996825,0.99229,0.992593



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.179974,0.019637,0.97619,0.997231,0.941634



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.016987,0.003727,0.987302,0.996644,0.97037



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.018499,0.002548,0.996825,0.992647,0.992593




organization              0.0
age                       0.0
cnt_encounters            0.0
cts                       0.0
contraceptive             0.0
anticoagulant             0.0
cnt_medications           0.0
cnt_procedures            0.0
last_encounter_dur        0.0
condition_dur             1.0
gender_M                  0.0
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### With SMOTE and using just features from original tables

In [ ]:
X_train_over_og = X_train_over[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()
X_test_og = X_test[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()

run_exps(X_train_over_og, y_train_over, X_test_og, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.007142,0.001774,0.465079,0.444786,0.249443



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.143075,0.015111,0.465079,0.444875,0.249443



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.013202,0.002395,0.457143,0.4407,0.246696



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.015541,0.001373,0.455556,0.438051,0.249453




gender_M                  0.28036415576934814
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.7196358442306519
encounterclass_outpatient 0.0


### With SMOTE and removing condition_dur feature

In [ ]:
X_train_over_c = X_train_over.drop('condition_dur', axis=1)
X_test_c = X_test.drop('condition_dur', axis=1)

run_exps(X_train_over_c, y_train_over, X_test_c, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.047239,0.001814,0.215873,0.92653,0.355091



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.142265,0.017754,0.842857,0.868734,0.642599



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.0203,0.008094,0.792063,0.850939,0.584127



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.018938,0.00248,0.796825,0.866687,0.592357




organization              0.13790105283260345
age                       0.057354945689439774
cnt_encounters            0.4425266683101654
cts                       0.0
contraceptive             0.0
anticoagulant             0.0
cnt_medications           0.0
cnt_procedures            0.0
last_encounter_dur        0.2958962321281433
gender_M                  0.029600033536553383
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.036721035838127136
encounterclass_outpatient 0.0
